In [1]:
#import and load data file
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intent.json').read()
intents = json.loads(data_file)

In [2]:
#preproccessdata
for intent in intents['intents']:
    for pattern in intent['text']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['intent']))

        # add to our classes list
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

143 documents
22 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
117 unique lemmatized words ["'s", ',', 'a', 'about', 'adam', 'adios', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'helpful', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identify', 'in', 'intelligent', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'op

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-4-451aefdaa425>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
29/29 [==============================] - 1s 2ms/step - loss: 3.1422 - accuracy: 0.0059
Epoch 2/200
29/29 [==============================] - 0s 2ms/step - loss: 3.0284 - accuracy: 0.0988
Epoch 3/200
29/29 [==============================] - 0s 2ms/step - loss: 2.9705 - accuracy: 0.1407
Epoch 4/200
29/29 [==============================] - 0s 2ms/step - loss: 2.7237 - accuracy: 0.2121
Epoch 5/200
29/29 [==============================] - 0s 1ms/step - loss: 2.5656 - accuracy: 0.3207
Epoch 6/200
29/29 [==============================] - 0s 2ms/step - loss: 2.3662 - accuracy: 0.3665
Epoch 7/200
29/29 [==============================] - 0s 1ms/step - loss: 1.9889 - accuracy: 0.4602
Epoch 8/200
29/29 [==============================] - ETA: 0s - loss: 1.8612 - accuracy: 0.20 - 0s 2ms/step - loss: 1.9691 - accuracy: 0.4384
Epoch 9/200
29/29 [==============================] - 0s 2ms/step - loss: 1.8463 - accuracy: 0.4487
Epoch 10/200
29/29 [==============================] - 0s 1ms/step -

29/29 [==============================] - 0s 1ms/step - loss: 0.0564 - accuracy: 0.9990
Epoch 83/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1109 - accuracy: 0.9779
Epoch 84/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1032 - accuracy: 0.9821
Epoch 85/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0806 - accuracy: 0.9905
Epoch 86/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1331 - accuracy: 0.9494
Epoch 87/200
29/29 [==============================] - 0s 1ms/step - loss: 0.2387 - accuracy: 0.8841
Epoch 88/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0816 - accuracy: 0.9835
Epoch 89/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1171 - accuracy: 0.9787
Epoch 90/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1046 - accuracy: 0.9668
Epoch 91/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1219 - accuracy: 0.9623
Epoch 92/200


29/29 [==============================] - 0s 1ms/step - loss: 0.0703 - accuracy: 0.9842
Epoch 164/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0892 - accuracy: 0.9724
Epoch 165/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0938 - accuracy: 0.9821
Epoch 166/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0405 - accuracy: 0.9966
Epoch 167/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0906 - accuracy: 0.9647
Epoch 168/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0951 - accuracy: 0.9660
Epoch 169/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0609 - accuracy: 0.9860
Epoch 170/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0455 - accuracy: 0.9829
Epoch 171/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0620 - accuracy: 0.9855
Epoch 172/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0729 - accuracy: 0.9675
Epoc

In [6]:
model.save('model.h5')

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intent.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [9]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['intents']== intents):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [11]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-11-896b278da15a>", line 15, in send
    res = chatbot_response(msg)
  File "<ipython-input-9-9f60469d7053>", line 12, in chatbot_response
    res = getResponse(ints, intents)
  File "<ipython-input-9-9f60469d7053>", line 5, in getResponse
    if(i['intents']== intents):
KeyError: 'intents'
